In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Download the dataset

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-17 13:18:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2023-07-17 13:18:36 (96.4 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [3]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of the tiny shakesphere dataset", len(text))

length of the tiny shakesphere dataset 1115394


In [5]:
# Checking the vocabulary size

vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print("Vocabulary: ", ''.join(vocab), "\nVocabulary_size", vocab_size)

Vocabulary:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 
Vocabulary_size 65


### Tokenize the data

Tokenizing the data actually means that encoding our input data which are in text into integers(numbers) which the computer can understand

- Google uses **Sentence-piece** tokenizer to encode the input data
- OpenAI uses **tiktoken** tokenizer to encode the input data

In [6]:
# Creating a mapping from charaters to integers
stoi = {ch:i for i,ch in enumerate(vocab)}
itos = {i:ch for i,ch in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[c] for c in l)

In [7]:
encoded = encode("hii there")
print(encoded)
decoded = decode(encoded)
print(decoded)

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# Encode the entire dataset and store it in tensor
import torch

encoded_data = encode(text)
data = torch.tensor(encoded_data, dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x7fb6caf331a0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [9]:
# Split into train data and val data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### Data Loader: Batches of chunk of data

In [10]:
# Visualize and understand the input and output

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context} the output is {target}")

When the input is tensor([18]) the output is 47
When the input is tensor([18, 47]) the output is 56
When the input is tensor([18, 47, 56]) the output is 57
When the input is tensor([18, 47, 56, 57]) the output is 58
When the input is tensor([18, 47, 56, 57, 58]) the output is 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the output is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the output is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the output is 58


In [12]:
torch.manual_seed(1729)
batch_size = 4 # How many sequence will we process in parallel?
block_size = 8 # What is the maximum context length for prediction?

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch("train")
print("inputs: ","\n", xb.shape,"\n", xb)
print("targets:", "\n", yb.shape, "\n", yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When the input is {context.tolist()} the output is {target}")
    

inputs:  
 torch.Size([4, 8]) 
 tensor([[57, 11,  0, 18, 53, 56,  1,  5],
        [40, 43,  1, 61, 43, 50, 50,  1],
        [52, 43, 56, 39, 50,  8,  0, 31],
        [46, 58,  1, 53, 44,  1, 57, 59]])
targets: 
 torch.Size([4, 8]) 
 tensor([[11,  0, 18, 53, 56,  1,  5, 58],
        [43,  1, 61, 43, 50, 50,  1, 39],
        [43, 56, 39, 50,  8,  0, 31, 46],
        [58,  1, 53, 44,  1, 57, 59, 41]])
When the input is [57] the output is 11
When the input is [57, 11] the output is 0
When the input is [57, 11, 0] the output is 18
When the input is [57, 11, 0, 18] the output is 53
When the input is [57, 11, 0, 18, 53] the output is 56
When the input is [57, 11, 0, 18, 53, 56] the output is 1
When the input is [57, 11, 0, 18, 53, 56, 1] the output is 5
When the input is [57, 11, 0, 18, 53, 56, 1, 5] the output is 58
When the input is [40] the output is 43
When the input is [40, 43] the output is 1
When the input is [40, 43, 1] the output is 61
When the input is [40, 43, 1, 61] the output is 

### Bigram Model

In [18]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1729)


class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits from the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        print(self.token_embedding_table)
        
    
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # Calculate the cross entropy loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            # Apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) #(B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            # append sampled index to the urunning sequence
            idx = torch.cat((idx, idx_next), dim =1) # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

Embedding(65, 65)
torch.Size([32, 65])
tensor(4.5080, grad_fn=<NllLossBackward0>)

'dsygtny:u&UeOdUoE!SZDoz ,r;XduqxqRfl$hAAPGQYJK'AjoyRBxu&S.Hf$ehumv;vkSfInJ.HnUeOkssEZ&&noAAA.iGpu&O


### Training the model

In [19]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [21]:
batch_size = 32
for steps in range(10000):
    
    # Sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    # Zeroing out all the gradients from the previous step
    optimizer.zero_grad(set_to_none=True)
    # Getting the parameters from all the gradients
    loss.backward()
    # Use the gradients to update the parameters
    optimizer.step()

print(loss.item())

2.6193814277648926


In [23]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=600)[0].tolist()))


SCHolar'd tofowee
waineerece f fan, me n thothilererndanstsheserd tllly vitchin:
YBEGHNon
CHerd wed s s?
HUSancoug PRCI Cl and d t;
LAs courte
ALAnpofamos ptaistaur'aurit honest d.
Dof ke c isher, s d ithe sld wiren irk the t:
L:
Thor halleishene ome anoun
y wites al s m,
ARI m,

INTevenoseered pa s, Ano--s th
AMES:

Cotheabre athb g, f:
May wis acly she wnd, hrey y antin aveng, t n:

Anind he;
Yewoufsly,
Mas athereyofly.
d.
Tfave bausem:
M:
Bre I an thonshad che hand w, ANRD be, sieayind bon a m bugepus Hedopeiss,
ARD: trt, cten the busn.

Ses:

PO: wir t t.
Her ou has tir'rqusthe sous as l.



### Self attention block

#### The mathematical trick in self-attention block

In [26]:
# Toy example

torch.manual_seed(1729)
B, T, C = 4, 8, 2 # Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [34]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0) # Doubt on what 0 does here??

In [36]:
# Version 2: Vectorizing the above operation
wei = torch.tril(torch.ones(T, T))
wei = wei/ torch.sum(wei, 1, keepdim = True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) = (B, T, C)
torch.allclose(xbow, xbow2)

True

In [39]:
# Version 3:
tril = torch.tril(torch.ones(T, T))
wei = torch.ones(T, T)
wei = tril.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [35]:
# The trick in self-attention: matrix multiply as weighted aggregation
torch.manual_seed(1729)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[1., 0.],
        [7., 7.],
        [9., 0.]])
---
c=
tensor([[1.0000, 0.0000],
        [4.0000, 3.5000],
        [5.6667, 2.3333]])


In [42]:
# version 4: Self-attention
torch.manual_seed(1729)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = key(x)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

**Notes:**
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode vectors
- Each example across batch dimension is of course processed completely independently and "never talk" to each other
- In an "encoder" attention block, just delete the single line that does masking with tril, allowing all tokens to communicate with each other. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in auto-regressive settings(GPT-2, GPT-3), like language modeling.
- "Self-attention" means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source(e.g. an encoder module)
- "Scaled attention" additionally divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and softmax will stay diffuse and not saturate too much.

In [50]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = k @ q.transpose(-2, -1) * head_size**-0.5

In [51]:
k.var()

tensor(0.9954)

In [52]:
q.var()

tensor(1.0377)

In [53]:
wei.var()

tensor(1.1652)

In [55]:
torch.softmax(torch.tensor([0.1, -0.2, -0.3, 0.4, 0.5]), dim=-1)

tensor([0.1904, 0.1410, 0.1276, 0.2570, 0.2840])

In [67]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])